In [2]:
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
sy.logger.remove()

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DOMAIN1_PORT = 8081
DOMAIN2_PORT = 8082
p1 =  sy.login(email="info@openmined.org",password="changethis",port=DOMAIN1_PORT)
p2 = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)
parties = [p1,p2]
t1 = sy.Tensor(np.array([1,2,3])).send(p1)
t2 = sy.Tensor(np.array([1,2,3])).send(p2)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!


kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/azureuser


In [12]:
#Creation of training data.
#Should modify to a concatenate type for pointer abstraction.

X = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])
y = np.array([[0,1,1,0]]).T


entity = Entity(name="rasswanth")
min_vals = np.array([0]*X.shape[0])
max_vals = np.array([1]*X.shape[0])

X = sy.Tensor(X)
X.public_shape = X.shape
X = X.private(
    0,3 , entities=[entity.name]*X.shape[0],ndept=True,
).tag("trade_flow")

d1 = X.send(p1)

# X = MPCTensor(secret=d1,parties=parties,shape=X.shape)

y = sy.Tensor(y)
y.public_shape = y.shape
y = y.private(
    0, 3, entities=[entity.name]*y.shape[0],ndept=True,
).tag("trade_flow")

d2 = y.send(p1)

# y = MPCTensor(secret=d2,parties=parties,shape=y.shape)



In [14]:
X.child.min_vals.to_numpy()

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [ ]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [ ]:

syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1
for j in range(1):
    l1 = relu(X @ syn0)  ; l1.block
    l2 = relu(l1 @ syn1) ; l2.block 
    l2_delta = (y - l2)* relu(l2,deriv=True) ; l2_delta.block
    l1_delta = (l2_delta@(syn1.T)) * relu(l1,deriv=True) ; l1_delta.block
    syn1  = syn1 + l1.T @ l2_delta ; syn1.block
    syn0 =  syn0 + X.T @ l1_delta ; syn0.block